In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

print("libraries uploaded")

In [ ]:
df = pd.read_excel(
    r'O:\Procurement Planning\QA\QA Reviews\Daily Medium-Dollar Invoice Report\2020\(12) December\12.17.2020 Daily Medium Dollar Invoice Report.xlsx')
df.head()

In [ ]:
# filter on previous day's date

df = df[(df['Invoice Submit Date - Date'] == '2020-12-16')]
print(len(df))
df.head()

In [ ]:
# Remove 'Unclassified' and re-format dates

df.replace('Unclassified',np.nan, inplace=True)

df['Invoice Submit Date - Date'] = pd.to_datetime(df['Invoice Submit Date - Date'])
df['Invoice Date Created - Date'] = pd.to_datetime(df['Invoice Date Created - Date'])
df['Invoice Date - Date'] = pd.to_datetime(df['Invoice Date - Date'])
df['Approved Date - Date'] = pd.to_datetime(df['Approved Date - Date'])
df.head()

In [ ]:
# Function to create the 'criteria met' column

def f(row):
    if row['sum(Invoice Spend)'] >= 10000 and row['sum(Invoice Spend)'] < 100000:
        val = 'Yes'
    else:
        val = 'No'
    
    return val

In [ ]:
# add the criteria met column

df['Criteria Met'] = df.apply(f,axis=1)
print(len(df))
df.head()

In [ ]:
# Filter on criteria met

df = df[(df['Criteria Met'] == 'Yes')]
print(len(df))
df

In [ ]:
# randomly sample data. 
# The IF argument checks if there are more or less than 10 rows. If there are 10 or fewer then Python selects distinct
# records based on Invoice ID. If there are more than 10 Python selects a random 10. 

if len(df) > 10:
    df = df.sample(10)
    

elif len(df) <= 10:
     df = df.groupby('Invoice ID').apply(lambda df: df.sample(1))
        

print(len(df))        
df

In [ ]:
df.drop('Criteria Met',axis=1, inplace = True)
print(len(df))

In [ ]:
# write df to sheet named 'Sample' in existing file

path = r'O:\Procurement Planning\QA\QA Reviews\Daily Medium-Dollar Invoice Report\2020\(12) December\12.17.2020 Daily Medium Dollar Invoice Report.xlsx'

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine='openpyxl')
writer.book = book
hide_sheet = book.get_sheet_names()
ws = book.get_sheet_by_name('Medium-Dollar Invoices for Audi')
ws.sheet_state = 'hidden'


df.to_excel(writer, sheet_name='Sample',index=False)


writer.save()
writer.close()
